# 常用卷积网络
网络复杂度有三个表征维度：深度、宽度、基数。VGG和残差网络最适合改进

#1 AlexNet
8层,ImageNet-2012竞赛第一名，标志这DNN(深度神经网络)深度学习革命的开始，它使用了5个卷积层和3个全连接层的网络结构，包含60M个参数，以及650K个神经元，使用了两个GPU。特点是加入了局部响应归一化(Local Response Normalization$\Rightarrow$LRN)，模拟神经元的侧抑制机制. 红色的数字表示输出通道数(卷积核的个数，也就是输出的层数)，天蓝色表示卷积核的厚度
- 输入：$224*224*3$
- 卷积层1：<span style='color:red'>96</span>$*11*11*$<span style='color:cyan'>3</span>, $\rightarrow{LRN}\rightarrow{MP}\rightarrow{ReLU}$, LRN：局部响应归一化，MP：最大池化，每个显卡是$\frac{96}{2}=48$。采用96个$11*11*3$的卷积核，在步长为4的情况下进行滤波，然后进行一次ReLU，一次Norm，一次Pooling，输出到下一层
- 卷积层2：<span style='color:red'>256</span>$*5*5*$<span style='color:cyan'>48</span>, $\rightarrow{LRN}\rightarrow{MP}\rightarrow{ReLU}$，(<span style='color:cyan'>通道独立</span>)
- 卷积层3：<span style='color:red'>384</span>$*3*3*$<span style='color:cyan'>256</span>, $\rightarrow{ReLU}$，(<span style='color:cyan'>通道合并，双GPU交互</span>)
- 卷积层4：<span style='color:red'>384</span>$*3*3*$<span style='color:cyan'>192</span>, $\rightarrow{MP}\rightarrow{ReLU}$，(<span style='color:cyan'>通道独立</span>)
- 卷积层5：<span style='color:red'>256</span>$*3*3*$<span style='color:cyan'>192</span>, $\rightarrow{ReLU}$，(<span style='color:cyan'>通道独立</span>)
- 全连接层1：$4096\rightarrow{ReLU}$
- 全连接层2：$4096\rightarrow{ReLU}$
- 全连接层3：$4096\rightarrow{ReLU}$
- Softmax层：$1000$
- Loss:标签概率的log值之和$argmax_{\omega}\{\frac{1}{N}\sum-log[p(f(w,\omega)=y(x))]\}$

#2 VGG
19层(一般有16层和19层两种，19层使用的比较多)，结构简洁，容易修改，容易迁移到其它任务中去.高层任务的基础网络，ImageNet2014年竞赛第二名.它的重要思想就是大卷积核分解成连续多个小卷积核，这样有两个好处
- 核分解：$7*7\rightarrow$3个$3*3$核(核与核之间用ReLU连接)(被$7*7$的卷积核卷积后，会丢失6个维度，被$3*3$的卷积核卷积后，会丢失2个维度，所以说$7*7$的卷积核就等于3个$3*3$的卷积核的级联  )
- 参数数量：$49C^2\rightarrow{27C^2}$，这样可以减少参数， ，增加深度

网络改造的首选基础网络，例如FCN，图片描述，图片问答等等，一样也是5个卷积层+3个全连接层，一样是两个GPU.![images](../images/12/28.png)

#3 GoogleLeNet
22层,ImageNet2014竞赛第一名，网络复杂，它的进化顺序是$InceptionV1\rightarrow{InceptionV2}\rightarrow{InceptionV3}\rightarrow{InceptionV4}$，其中Inception V1跟VGG是同一时期的,Inception V3和INception V4和ResNet残差网络是同一时期的。这个网络为了提升性能，减少了参数，降低了计算，但是增加了宽度和深度

#4 ResNet-残差网络
152层，结构简洁，容易修改，容易迁移到其它任务中去.高层任务的基础网络![images](../images/12/29.png)下图左侧是普通的卷积，右图增加了一条直连线，所以最后的输出变成了$H(x)=F(x)+x$，可以拟合出任意目标映射,H(x)就是残差块，可以保证模型更深，收敛更快，梯度损失的更慢
- 全是$3*3$卷积核
- 卷积步长为2，可以取代池化
- 取消了池化、全连接和Dropout操作(全连接层用到了1层，并不是完全取消了)

![images](../images/12/30.png)

#5 ResNeXt网络
创造型的提出了DNN的第三个维度:Cardinality基数维度，采用了Split-Transform-Aggregate策略，将卷积核按通道分组，形成32个并行分支，低纬度卷积进行特征变换，加法合并。在同参数规模下，增加结构，提高了模型表达力。100层的ResNeXt=200层ResNet。ILSVRC2016年竞赛第二名![images](../images/12/31.png)左边是传统的残差网络，右边是ResNet残差网络。上图中，ResNet卷积是256通道输入，通过Bootlenet降维成$1*1*64$，然后进行$3*3*64$的卷积，然后在升维成$1*1*256$。对于ResNeXt来说，有32个分支，每个分支上做的跟ResNet一样，最后就有32个256个通道的输出，然后这32个256的输出，做加和。但是这32个通道的参数是不一样的，需要各自学习

#6 卷积神经网络的比较
![images](../images/12/32.png)

#7 技巧

##7.1 避免信息瓶颈
在卷积过程中，空间尺寸$H*W$会变小，输出通道数C会变多，数据量$H*W*C$要缓慢变小

##7.2 通道数量保持在可控范围内
卷积核尺寸不能太大，通道数量也不能太大，因为参数的数量是随着卷积核尺寸和通道数量成指数变化的，比如如果输入通道数量为C，输出通道数量为H，那么
- 参数数量$H_f*W_f*C*K$
- 操作数量$\frac{H*H_f}{stride}*\frac{W*W_f}{stride}*C*K$

##7.3 感受野要足够大
卷积是基于局部图片的操作，可以捕捉大尺寸内容，所以需要把一个大卷积，变成若干个小卷积，这样可以减少参数，计算快，还可以增加多个非线性激活

##7.4 通过分组策略降低计算量
分组策略包括通道拆解，分GPU，等等![images](../image/12/33.png)

##7.5 通过低秩分解，降低参数和计算量
![images](../images/12/34.png)

#8 目标分类的基本框架

##8.1 数据准备
- 数据来源：现有数据集的子集；网络采集；现有数据人工标注
- 数据补充：原始数据切割；噪声颜色等像素变化；旋转平移等姿态变化
- 数据规范：均值处理；归一化；大小调整

##8.2 模型设计
- 任务类型：分类(表情分类，人群分类)、分类+回归(表情+程度，种类+信心，什么人+人数)、多目标分类(面部行为、群体行为、车流预测)
- 现有模型：已有的方案能否(怎样)借鉴
- 局部更改或从头设计：从哪里改变，如何确定新参数，新结构的特点

##8.3 训练细节
- GPU-Batch size，是否并行
- 数据循环方式、平衡性考虑
- 网络深度宽度确定
- 损失函数设计
- 学习率变化方式，模型各层学习率是否一致
- 评价方式：准确率，F1 score

#9 如何设计神经网络-面部识别(AU detection)